In [19]:
import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon
import skimage.io
from pathlib import Path
from loguru import logger


In [51]:
class AnnotationTOmask:
    def __init__(self, annotation_file):
        # load dataset
        print("loading annotations ...")
        dataset = json.load(open(annotation_file, "r"))
        print("annotations loaded!")

        # creating index
        print("creating index...")
        imgToAnns = {ann["image_id"]: [] for ann in dataset["annotations"]}
        anns = {ann["id"]: [] for ann in dataset["annotations"]}
        for ann in dataset["annotations"]:
            imgToAnns[ann["image_id"]] += [ann]
            anns[ann["id"]] = ann

        imgs = {im["id"]: {} for im in dataset["images"]}
        for img in dataset["images"]:
            imgs[img["id"]] = img

        cats = []
        catToImgs = []
        cats = {cat["id"]: [] for cat in dataset["categories"]}
        for cat in dataset["categories"]:
            cats[cat["id"]] = cat
        catToImgs = {cat["id"]: [] for cat in dataset["categories"]}
        for ann in dataset["annotations"]:
            catToImgs[ann["category_id"]] += [ann["image_id"]]

        print(f"index created! {len(imgs)} images, {len(anns)} annotations, {len(cats)} categories")

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.catToImgs = catToImgs
        self.imgs = imgs
        self.cats = cats
        self.dataset = dataset

    def getAnnIds(self, imgIds, catIds):
        """
        Get ann ids for given cats from all images
        """
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        catIds = catIds if type(catIds) == list else [catIds]

        if not len(imgIds) == 0:
            anns = sum(
                [self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns],
                [],
            )
        else:
            anns = self.dataset["annotations"]
        anns = (
            anns
            if len(catIds) == 0
            else [ann for ann in anns if ann["category_id"] in catIds]
        )
        ids = [ann["id"] for ann in anns]

        return ids

    def getCatIds(self, catNms=[], catIds=[]):
        """
        get integer array of cat ids for given cat names, given cat ids
        """
        if len(catNms) == len(catIds) == 0:
            cats = self.dataset["categories"]
        else:
            print("+++++++++++++++++++++")
            cats = self.dataset["categories"]
            cats = (
                cats
                if len(catNms) == 0
                else [cat for cat in cats if cat["name"] in catNms]
            )
            cats = (
                cats
                if len(catIds) == 0
                else [cat for cat in cats if cat["id"] in catIds]
            )
        ids = [cat["id"] for cat in cats]
        return ids

    def getImgIds(self, imgIds=[], catIds=[]):
        """
        return an integer array of img ids for given ids with all given cats
        """
        ids = set(imgIds)
        for catId in catIds:
            if len(ids) == 0:
                ids = set(self.catToImgs[catId])
            else:
                ids &= set(self.catToImgs[catId])
        return list(ids)

    def loadAnns(self, ids=[]):
        """
        loaded ann objects for integer ids specifying annotations
        """
        if type(ids) == list:
            return [self.anns[id] for id in ids]
        elif type(ids) == int:
            return [self.anns[ids]]

    def loadImgs(self, ids):
        """
        Load loaded img objects with the specified ids.
        """
        return [self.imgs[ids]]

    def getSeg(self, anns):
        """
        get annotations segmentatins
        """
        if len(anns) == 0:
            print("no annotations found")
            return 0

        S = []
        for ann in anns:
            for seg in ann["segmentation"]:
                S.append(seg)
        return S

    def segToMask(self, S, h, w):
        """
        Convert polygon segmentation to binary mask.
          S: polygon segmentation mask, h: target mask height, w: target mask width
        """
        M = np.zeros((h, w))
        for s in S:
            N = len(s)
            rr, cc = polygon(np.array(s[1:N:2]), np.array(s[0:N:2]))  # (y, x)
            M[rr, cc] = 1
        return M


def coco_to_mask(
    coco_filename,
    output_dir,
    organ,
    voxelsize_mm=None,
    output_type="png",
    show=False,
    name_prefix="",
):
    """
    :param coco_filename: coco_filename must include not only name of Coco file, but also it`s full direction (location) in your PC
    :param output_dir: is used for controll output direction (location) in your PC
    :param organ: name of segmentation part rom what we want have mask ""
    :param voxelsize_mm:
    :param output_type: type of "Save fail" of our program| results]
    :param show:
    :return:
    """
    # file_path = 'task_cell track 20200226-dii-30las-2pre1-2020_10_26_13_28_36-coco 1.0/annotations/instances_default.json'

    file_path = coco_filename
    import datetime

    t0 = datetime.datetime.now()
    cv_an = AnnotationTOmask(file_path)
    t1 = datetime.datetime.now()
    cv_an1 = cv_an.getImgIds()
    t2 = datetime.datetime.now()
    logger.debug(f"{t1 - t0}, {t2 - t1}")
    logger.debug("getImgIds", cv_an1)
    # extract the region we want to mask ( Right Kidny,  Liver)

    # catIds = cv_an.getCatIds(catNms=['cell'])
    catIds = cv_an.getCatIds(catNms=[organ])
    # catIds = cv_an.getCatIds(catNms=['Liver'])
    # catIds = cv_an.getCatIds(catNms=['Left Kidney'])

    # logger.debug("catIds", catIds)
    imgIds = cv_an.getImgIds(catIds=catIds)
    if len(imgIds) == 0:
        logger.warning(f"Label '{organ}' not found.")
    # logger.debug(f"imgIds={imgIds}")
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # TODO allow storing into one file
    #  imgName = cv_an.dataset['images'][0]
    #  M = np.zeros([imgName['width'], imgName['height'],len(cv_an["images"])])
    #  ...

    # create empty files
    for imgName in cv_an.dataset["images"]:
        # logger.debug(f"imgName={imgName}")
        M = np.zeros([imgName["width"], imgName["height"]])
        image_path = Path(output_dir) / organ / (
            name_prefix + 
            # Path(imgName["file_name"]).name + "." + output_type
            imgName["file_name"] + "." + output_type
        )
        image_path.parent.mkdir(parents=True, exist_ok=True)
        # logger.debug(f"image_path={image_path}")
        skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
        # plt.imsave(image_path, np.uint8(M), cmap = 'gray')

    # rewrite images with label
    for im in tqdm.tqdm(imgIds):
        # logger.debug(f"iM{im}")
        imgName = cv_an.loadImgs(im)[0]
        anns_ids = cv_an.getAnnIds(imgIds=imgName["id"], catIds=catIds)
        anns = cv_an.loadAnns(anns_ids)
        S = cv_an.getSeg(anns)
        M = cv_an.segToMask(S, imgName["width"], imgName["height"]) * 255
        if show:
            plt.figure()
            plt.imshow(M)
            plt.show()
            plt.close()
        # image_path = Path(output_dir) / ('MaskOfPIG_'+str(imgName['file_name'])+'.'+output_type)
        image_path = Path(output_dir) / organ / (
                name_prefix +
                # Path(imgName["file_name"]).name + "." + output_type
                imgName["file_name"] + "." + output_type
        )
        # logger.debug(image_path)
        un = np.unique(M)
        if len(un) < 2:
            logger.warning(f"No data found in annotation {imgName['file_name']}")
        skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
        plt.imsave(image_path, np.uint8(M), cmap="gray")

    logger.debug("Done")

In [15]:
# json_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\pilsen_pigs_coco_from_cvat/instances_default.json")
# # json_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\tx028_coco\annotations\instances_default.json")
# assert json_path.exists()

In [16]:

# file_path = json_path
# organ="Left Kidney"
# 
# 
# import datetime
# 
# t0 = datetime.datetime.now()
# cv_an = AnnotationTOmask(file_path)
# t1 = datetime.datetime.now()
# cv_an1 = cv_an.getImgIds()
# t2 = datetime.datetime.now()
# logger.debug(f"{t1 - t0}, {t2 - t1}")
# logger.debug("getImgIds", cv_an1)
# # extract the region we want to mask ( Right Kidny,  Liver)
# 
# # catIds = cv_an.getCatIds(catNms=['cell'])
# catIds = cv_an.getCatIds(catNms=[organ])
# # catIds = cv_an.getCatIds(catNms=['Liver'])
# # catIds = cv_an.getCatIds(catNms=['Left Kidney'])
# 
# logger.debug("catIds", catIds)
# imgIds = cv_an.getImgIds(catIds=catIds)
# if len(imgIds) == 0:
#     logger.warning(f"Label '{organ}' not found.")


2024-07-29 10:26:25.423 | DEBUG    | __main__:<module>:12 - 0:00:00.143986, 0:00:00
2024-07-29 10:26:25.424 | DEBUG    | __main__:<module>:13 - getImgIds
2024-07-29 10:26:25.424 | DEBUG    | __main__:<module>:21 - catIds


loading annotations ...
annotations loaded!
creating index...
index created!
+++++++++++++++++++++


In [25]:
# len(cv_an.dataset["images"])
cv_an.dataset["images"][0]


{'id': 1,
 'width': 512,
 'height': 512,
 'file_name': 'Tx040D_Ven/Tx040D_Vensoubor_01123.jpg',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0}

In [24]:
# cv_an.dataset["categories"]

[{'id': 1, 'name': 'Heart', 'supercategory': ''},
 {'id': 2, 'name': 'Thoracic Cavity', 'supercategory': ''},
 {'id': 3, 'name': 'Left Kidney', 'supercategory': ''},
 {'id': 4, 'name': 'Right Kidney', 'supercategory': ''},
 {'id': 5, 'name': 'Liver', 'supercategory': ''},
 {'id': 6, 'name': 'Portal Vein', 'supercategory': ''},
 {'id': 7, 'name': 'Vena Cava', 'supercategory': ''},
 {'id': 8, 'name': 'Aorta', 'supercategory': ''},
 {'id': 9, 'name': 'Portal Vein Entry Point', 'supercategory': ''}]

In [40]:

coco_path= Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\pilsen_pigs_coco_from_cvat/instances_default.json")
output_path_png= Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\masks_from_annotations\2024-07_png")
output_path_mhd = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\masks_from_annotations\2024-07_mhd")


In [52]:
# Coco_Path = "F:/PigProject/pig28/annotations/Pig28.json"
# Output_Path = "F:/PigProject/pig28/annotations/PNG"
# organ = "Left Kidney"

for organ in [
    # "Left Kidney", 
    "Right Kidney", "Liver", "Portal Vein", "Aorta"
]:
    coco_to_mask1 = coco_to_mask(
        coco_path,
        output_path_png,
        organ,
        voxelsize_mm=None,
        output_type="png",
        show=False,
        name_prefix="",
    )

loading annotations ...


2024-07-29 11:13:59.846 | DEBUG    | __main__:coco_to_mask:166 - 0:00:00.230001, 0:00:00
2024-07-29 11:13:59.847 | DEBUG    | __main__:coco_to_mask:167 - getImgIds


annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories
+++++++++++++++++++++


100%|██████████| 1006/1006 [00:25<00:00, 38.86it/s]
2024-07-29 11:15:20.664 | DEBUG    | __main__:coco_to_mask:227 - Done
2024-07-29 11:15:20.823 | DEBUG    | __main__:coco_to_mask:166 - 0:00:00.141998, 0:00:00
2024-07-29 11:15:20.824 | DEBUG    | __main__:coco_to_mask:167 - getImgIds


loading annotations ...
annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories
+++++++++++++++++++++


...

2024-07-29 11:17:30.533 | DEBUG    | __main__:coco_to_mask:166 - 0:00:00.219023, 0:00:00
2024-07-29 11:17:30.534 | DEBUG    | __main__:coco_to_mask:167 - getImgIds


annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories
+++++++++++++++++++++


100%|██████████| 107/107 [00:02<00:00, 42.03it/s]
2024-07-29 11:18:38.571 | DEBUG    | __main__:coco_to_mask:227 - Done
2024-07-29 11:18:38.730 | DEBUG    | __main__:coco_to_mask:166 - 0:00:00.142999, 0:00:00
2024-07-29 11:18:38.731 | DEBUG    | __main__:coco_to_mask:167 - getImgIds
2024-07-29 11:18:38.731 | WARNING  | __main__:coco_to_mask:178 - Label 'Aorta' not found.


loading annotations ...
annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories
+++++++++++++++++++++


0it [00:00, ?it/s]
2024-07-29 11:19:33.071 | DEBUG    | __main__:coco_to_mask:227 - Done


# Convert to RAW / MHD

In [53]:
# go over dirs in output_path
import io3d


for fn in tqdm.tqdm(list(output_path_png.glob("**/*0000.jpg.png"))):
    fn_parent = fn.parent
    fn_parent_rel = fn_parent.relative_to(output_path_png)
    mhd_path = output_path_mhd / fn_parent_rel.parent / (fn_parent_rel.name + ".mhd")
    mhd_path.parent.mkdir(parents=True, exist_ok=True)
    # logger.debug(fn_parent_rel)
    # logger.debug(mhd_path)
    datap = io3d.read(fn_parent)
    io3d.write(datap.data3d, mhd_path, metadata={"voxelsize_mm": datap.voxelsize_mm})
    # logger.debug(f"{datap.data3d.shape}")
    
    

100%|██████████| 80/80 [06:42<00:00,  5.04s/it]
